# Draft PILLS Snapshot

This notebook combines historical PILLS balances into a single data frame, including Liquid Driver, Beets, and the 2 most recent Neo Pools.
These values are explained in a separate notebook.

The notebook also includes PILLS balances, current as of December 6, 2022.

In [1]:
import os
import json
import time
from pprint import pprint
from dotenv import load_dotenv
import pandas as pd
import web3

load_dotenv(os.path.expanduser('~/Work/morpheus-swap/.env'))

True

## Neo Pool contract addresses

In [2]:
with open(os.path.join(os.environ['DATA_PATH'], 'neo-pools.json'), 'r') as f:
    neo_pool_addresses = json.load(f)
neo_pool_addresses.reverse()
neo_pool_addresses = [ i['address'] for i in neo_pool_addresses ]
pprint(neo_pool_addresses[:3])

['0x5bcb5f2ed10ac292c9e281c5ead4f0533666c3b6',
 '0x326A7D91B6F767458599E7d93DFc850d2e083d30',
 '0xA431fDd6d23f0d7C4b4B92F4aD866a5939b53abB']


## Contract balances and call() results

In [3]:
df_contracts = pd.read_csv(os.path.join(os.environ['DATA_PATH'], '2022-04-20-pills-contracts.csv.gz'))
df_contracts.rename(columns={'pills_erc20_balance': 'pills_erc20_balance_historical'}, inplace=True)
df_contracts.head(1)

,address,pills_erc20_balance_historical,beets_lp_erc20_balance,neo_pool_0_amount,neo_pool_0_reward_debt,neo_pool_1_amount,neo_pool_1_reward_debt,spirit_amount,spirit_reward_debt
0,0x9C775D3D66167685B2A3F4567B548567D2875350,2.415931e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Merge in LQDR contract results

In [4]:
df_lqdr = pd.read_csv(os.path.join(os.environ['DATA_PATH'], 'lqdr-pills-gauge-balance.csv'))
del df_lqdr['Unnamed: 0']
df_lqdr.rename(columns={'balance': 'lqdr_lp_amount'}, inplace=True)
df_lqdr.head(1)
df_snapshot = pd.merge(df_contracts, df_lqdr, on='address')
df_snapshot.head(1)

,address,pills_erc20_balance_historical,beets_lp_erc20_balance,neo_pool_0_amount,neo_pool_0_reward_debt,neo_pool_1_amount,neo_pool_1_reward_debt,spirit_amount,spirit_reward_debt,lqdr_lp_amount
0,0x9C775D3D66167685B2A3F4567B548567D2875350,2.415931e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Unpack current PILLS balances from chaino parallel tasks

In [5]:
filename = os.path.join(os.environ['DATA_PATH'], '2022-12-17-pills-balances-parallel.json')

with open(filename, 'r') as f:
    results_parallel = json.load(f)

results_decimal = {}
for task_id in results_parallel.keys():
    results_decimal.update(results_parallel[task_id])

results = dict([ (k,v/10**18) for k, v in results_decimal.items() ])

csv_filename = os.path.join(os.environ['DATA_PATH'], '2022-12-17-pills-balances.csv')
df_current = pd.DataFrame(results.items(), columns=['address', 'pills_erc20_balance_current'])
df_current.head(1)

,address,pills_erc20_balance_current
0,0x9C775D3D66167685B2A3F4567B548567D2875350,281012.897854


## Transform LP to PILLS

In [6]:
csv_filename = os.path.join(os.environ['DATA_PATH'], 'pills-snapshot-draft.csv')
lqdr_pills_per_lp = 70.2112453400694
beets_pills_per_lp = 0.834235125170505

df_snapshot = pd.merge(df_snapshot, df_current, on='address')
df_snapshot['neo_pools_balance'] = df_snapshot['neo_pool_0_amount'] + df_snapshot['neo_pool_1_amount']
df_snapshot['lqdr_pills_balance'] = df_snapshot['lqdr_lp_amount'] * lqdr_pills_per_lp
df_snapshot['beets_pills_balance'] = df_snapshot['beets_lp_erc20_balance'] * beets_pills_per_lp

del df_snapshot['beets_lp_erc20_balance']
del df_snapshot['lqdr_lp_amount']
del df_snapshot['neo_pool_0_amount']
del df_snapshot['neo_pool_0_reward_debt']
del df_snapshot['neo_pool_1_amount']
del df_snapshot['neo_pool_1_reward_debt']
del df_snapshot['spirit_amount']
del df_snapshot['spirit_reward_debt']

df_snapshot.to_csv(csv_filename)
df_snapshot.head(5)

,address,pills_erc20_balance_historical,pills_erc20_balance_current,neo_pools_balance,lqdr_pills_balance,beets_pills_balance
0,0x9C775D3D66167685B2A3F4567B548567D2875350,2.415931e+06,2.810129e+05,0.000000,0.0,0.0
1,0xF04588329c3104779796F279A9df3803Ae83dB7B,0.000000e+00,5.150627e+03,50659.894950,0.0,0.0
2,0x326A7D91B6F767458599E7d93DFc850d2e083d30,2.882333e+06,1.999716e+05,0.000000,0.0,0.0
3,0x2728e32174FC95De00695007c9134fA03dC8b434,0.000000e+00,0.000000e+00,53376.318198,0.0,0.0
4,0xD488F4585Ab28D003324253be5D791aC103c0b57,8.730000e-16,8.730000e-16,0.000000,0.0,0.0


## Check that wallet total matches Sankey

The two most recent Neo pools were included in the snapshot.  A total of 336k pills were staked in other neo pools at the time of the snapshot.

In [7]:
other_neo_pool_total = 336780.609907302

df = df_snapshot.copy()
wallet_total = df['pills_erc20_balance_historical'].sum() - \
    df['neo_pools_balance'].sum() - \
    df['lqdr_pills_balance'].sum() - \
    df['beets_pills_balance'].sum() - \
    other_neo_pool_total
wallet_total

850364.1993269545